## Calculate properties of coaxial line

Properties of a coaxial line are calculated in the `coax` function. It returns multiple values in a namedtuple.

In [1]:
import numpy as np
from collections import namedtuple

import suncal
from suncal.unitmgr import ureg


In [2]:
mu0 = 1.25663706212E-6 * ureg.henry / ureg.meter  # H/m
eps0 = 8.8541878128E-12 * ureg.farad / ureg.meter # F/m
c = 299792458 * ureg.meter/ureg.second            # m/s

def coax(a, b, freq, rho, mu_r, eps_r, TL):
    ''' Calculate properties of coaxial line '''
    omega = 2 * np.pi * freq
    L = mu0 * mu_r / 2 / np.pi * np.log(b/a)   # H/m
    C = 2 * np.pi * eps0 * eps_r / np.log(b/a) # F/m
    R = np.sqrt(rho * freq * mu0 * mu_r / np.pi) * (1/a + 1/b) / 2   # ohm/m
    G = TL * np.pi * 2 * freq * C   # S/m
    Rdc = rho / np.pi / a**2        # DC resistance, ohm/m
    propvel = 1/np.sqrt(L*C)/c      # propagation velocity as fraction of c
    Z0 = abs(np.sqrt((R + 1j*omega*L)/(G + 1j*omega*C)))      # ohm
    atten = np.sqrt((R + 1j*omega*L)*(G + 1j*omega*C)).real   # Np/m

    coaxresult = namedtuple('Coax', ['L', 'C', 'R', 'G', 'Rdc', 'velocity', 'Z0', 'gamma'])
    return coaxresult(L, C, R, G, Rdc, propvel, Z0, atten)

In [3]:
# Note: changes in Pint broke conversion of last output to dB/hectofoot.
# For now, leaving it as 1/hectofoot.

uc = suncal.UncertaintyCalc(coax, units=['uH/m', 'pF/ft', 'ohm/m', '1/ohm/m', 'ohm/hectofeet',
                                         'dimensionless', 'ohm', '1/hectofoot'])
uc.set_input('a', 0.089/2, unc='1%', units='inch')            # inner radius
uc.set_input('b', 0.285/2, unc='1%', units='inch')            # outer radius
uc.set_input('freq', 100, units='MHz')                        # frequency
uc.set_input('rho', 2.21E-8, unc='1%', units='ohm*m')         # effective resistivity of center conductor (copper)
uc.set_input('mu_r', 1, units='dimensionless')                # permeability
uc.set_input('eps_r', 2.115, unc='1%', units='dimensionless') # permittivity of PE that results in 30.8 pF/ft
uc.set_input('TL', .0002, unc='1%', units='dimensionless')    # Loss tangent of PE (depends on frequency - this value is for 100 MHz)
uc.calculate().gum

|Function   | Nominal   | Std. Uncertainty  |
|----------|---------|------------------|
|$L$   | 0.2328 µH/m  | 0.0028 µH/m          |
|$C$   | 30.81 pF/ft  | 0.48 pF/ft          |
|$R$   | 0.5458 Ω/m  | 0.0051 Ω/m          |
|$G$   | 0.00001270 1/m/Ω  | 2.4e-07 1/m/Ω          |
|$Rdc$   | 0.1678 Ω/hft  | 0.0038 Ω/hft          |
|$velocity$   | 0.6876  | 0.0034          |
|$Z_{0}$   | 47.98 Ω | 0.63 Ω         |
|$\gamma$   | 0.1826 1/hft | 0.0023 1/hft         |

In [4]:
uc.out.gum.report_sens()

### L


|Variable   | Sensitivity   | Proportion  |
|----------|-------------|------------|
|$TL$   | 0.0 H/m      | 0.00%       |
|$a$   | -1.1e-07 H/in²      | 50.00%      |
|$b$   | 3.6e-08 H/in²      | 50.00%      |
|$eps_{r}$   | 0.0 H/m      | 0.00%       |
|$freq$   | 0.0 H/MHz/m      | 0.00%       |
|$\mu_{r}$   | 2.3e-07 H/m      | 0.00%       |
|$\rho$   | 0.0 H/m²/Ω      | 0.00%       |


### C


|Variable   | Sensitivity   | Proportion  |
|----------|-------------|------------|
|$TL$   | 0.0 F/m      | 0.00%       |
|$a$   | 5.0e-11 F/in²      | 29.81%      |
|$b$   | -1.5e-11 F/in²      | 29.81%      |
|$eps_{r}$  | 4.8e-11 F/m     | 40.38%      |
|$freq$  | 0.0 F/MHz/m     | 0.00%       |
|$\mu_{r}$  | 0.0 F/m     | 0.00%       |
|$\rho$  | 0.0 F/m²/Ω     | 0.00%       |


### R


|Variable   | Sensitivity   | Proportion  |
|----------|-------------|------------|
|$TL$  | 0.0 H⁰⋅⁵·MHz⁰⋅⁵·Ω⁰⋅⁵/in     | 0.00%       |
|$a$  | -0.00024 H⁰⋅⁵·MHz⁰⋅⁵·Ω⁰⋅⁵/in²     | 65.44%      |
|$b$  | -0.000023 H⁰⋅⁵·MHz⁰⋅⁵·Ω⁰⋅⁵/in²     | 6.38%       |
|$eps_{r}$  | 0.0 H⁰⋅⁵·MHz⁰⋅⁵·Ω⁰⋅⁵/in     | 0.00%       |
|$freq$  | 6.9e-08 H⁰⋅⁵·Ω⁰⋅⁵/MHz⁰⋅⁵/in     | 0.00%       |
|$\mu_{r}$  | 6.9e-06 H⁰⋅⁵·MHz⁰⋅⁵·Ω⁰⋅⁵/in     | 0.00%       |
|$\rho$  | 12348.3 H⁰⋅⁵·MHz⁰⋅⁵/m²/Ω⁰⋅⁵     | 28.17%      |


### G


|Variable   | Sensitivity   | Proportion  |
|----------|-------------|------------|
|$TL$  | 6.4e-08 F·MHz/m     | 28.76%      |
|$a$  | 6.2e-12 F·MHz/in²     | 21.24%      |
|$b$  | -1.9e-12 F·MHz/in²     | 21.24%      |
|$eps_{r}$  | 6.0e-12 F·MHz/m     | 28.76%      |
|$freq$  | 1.3e-13 F/m     | 0.00%       |
|$\mu_{r}$  | 0.0 F·MHz/m     | 0.00%       |
|$\rho$  | 0.0 F·MHz/m²/Ω     | 0.00%       |


### Rdc


|Variable   | Sensitivity   | Proportion  |
|----------|-------------|------------|
|$TL$  | 0.0 Ω/in     | 0.00%       |
|$a$  | -0.0063 Ω/in²     | 80.00%      |
|$b$  | 0.0 Ω/in²     | 0.00%       |
|$eps_{r}$  | 0.0 Ω/in     | 0.00%       |
|$freq$  | 0.0 Ω/MHz/in     | 0.00%       |
|$\mu_{r}$  | 0.0 Ω/in     | 0.00%       |
|$\rho$  | 160.7 1/in²     | 20.00%      |


### velocity


|Variable   | Sensitivity   | Proportion  |
|----------|-------------|------------|
|$TL$  | 0.0      | 0.00%       |
|$a$  | -2.5e-07 s/F⁰⋅⁵/H⁰⋅⁵/in     | 0.00%       |
|$b$  | 0.0 s/F⁰⋅⁵/H⁰⋅⁵/in     | 0.00%       |
|$eps_{r}$  | -0.16      | 100.00%     |
|$freq$  | 0.0 1/F⁰⋅⁵/H⁰⋅⁵/MHz²     | 0.00%       |
|$\mu_{r}$  | -0.34      | 0.00%       |
|$\rho$  | 0.0 s/F⁰⋅⁵/H⁰⋅⁵/m/Ω     | 0.00%       |


### Z0


|Variable   | Sensitivity   | Proportion  |
|----------|-------------|------------|
|$TL$  | -0.089 H⁰⋅²⁵·Ω⁰⋅²⁵/F⁰⋅⁵/MHz⁰⋅²⁵     | 0.00%       |
|$a$  | -1.1534611e+06 H⁰⋅²⁵·Ω⁰⋅²⁵/F⁰⋅⁵/MHz⁰⋅²⁵/m     | 42.76%      |
|$b$  | 3.602007e+05 H⁰⋅²⁵·Ω⁰⋅²⁵/F⁰⋅⁵/MHz⁰⋅²⁵/m     | 42.76%      |
|$eps_{r}$  | -358.7 H⁰⋅²⁵·Ω⁰⋅²⁵/F⁰⋅⁵/MHz⁰⋅²⁵     | 14.48%      |
|$freq$  | -0.000053 H⁰⋅²⁵·Ω⁰⋅²⁵/F⁰⋅⁵/MHz¹⋅²⁵     | 0.00%       |
|$\mu_{r}$  | 758.7 H⁰⋅²⁵·Ω⁰⋅²⁵/F⁰⋅⁵/MHz⁰⋅²⁵     | 0.00%       |
|$\rho$  | 2.392576e+05 H⁰⋅²⁵/F⁰⋅⁵/MHz⁰⋅²⁵/m/Ω⁰⋅⁷⁵     | 0.00%       |


### gamma


|Variable   | Sensitivity   | Proportion  |
|----------|-------------|------------|
|$TL$  | 0.000048 F⁰⋅⁵·H⁰⋅²⁵·MHz⁰⋅⁷⁵·Ω⁰⋅²⁵/m     | 0.16%       |
|$a$  | 0.000015 F⁰⋅⁵·H⁰⋅²⁵·MHz⁰⋅⁷⁵·Ω⁰⋅²⁵/m²     | 0.54%       |
|$b$  | -0.000055 F⁰⋅⁵·H⁰⋅²⁵·MHz⁰⋅⁷⁵·Ω⁰⋅²⁵/m²     | 69.04%      |
|$eps_{r}$  | 4.5e-08 F⁰⋅⁵·H⁰⋅²⁵·MHz⁰⋅⁷⁵·Ω⁰⋅²⁵/m     | 15.92%      |
|$freq$  | 1.0e-09 F⁰⋅⁵·H⁰⋅²⁵·Ω⁰⋅²⁵/MHz⁰⋅²⁵/m     | 0.00%       |
|$\mu_{r}$  | 4.8e-09 F⁰⋅⁵·H⁰⋅²⁵·MHz⁰⋅⁷⁵·Ω⁰⋅²⁵/m     | 0.00%       |
|$\rho$  | 4.1 F⁰⋅⁵·H⁰⋅²⁵·MHz⁰⋅⁷⁵/m²/Ω⁰⋅⁷⁵     | 14.34%      |